In [1]:
import pandas as pd
import string
import nltk
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from queue import PriorityQueue as pq
import editdistance
from nltk.corpus import words
import cohere

In [2]:
adv_reads = pd.read_csv('google_books_1299.csv')
adv_reads.drop(columns=adv_reads.columns[0], axis=1, inplace=True)

In [3]:
adv_reads.drop_duplicates(keep='first', inplace=True, ignore_index=True)

# manually set: should we do this?
adv_reads['lower_age'] = [16 for i in range(len(adv_reads))]
adv_reads['upper_age'] = [20 for i in range(len(adv_reads))]
adv_reads.head(2000)


,title,author,rating,voters,price,currency,description,publisher,page_count,generes,ISBN,language,published_date,lower_age,upper_age
0,Attack on Titan: Volume 13,Hajime Isayama,4.6,428,43.28,SAR,NO SAFE PLACE LEFT At great cost to the Garris...,Kodansha Comics,192,none,9781612626864,English,"Jul 31, 2014",16,20
1,Antiques Roadkill: A Trash 'n' Treasures Mystery,Barbara Allan,3.3,23,26.15,SAR,Determined to make a new start in her quaint h...,Kensington Publishing Corp.,288,"Fiction , Mystery &amp, Detective , Cozy , Gen...",9780758272799,English,"Jul 1, 2007",16,20
2,The Art of Super Mario Odyssey,Nintendo,3.9,9,133.85,SAR,Take a globetrotting journey all over the worl...,Dark Horse Comics,368,"Games &amp, Activities , Video &amp, Electronic",9781506713816,English,"Nov 5, 2019",16,20
3,Getting Away Is Deadly: An Ellie Avery Mystery,Sara Rosett,4.0,10,26.15,SAR,"With swollen feet and swelling belly, pregnant...",Kensington Publishing Corp.,320,none,9781617734076,English,"Mar 1, 2009",16,20
4,"The Painted Man (The Demon Cycle, Book 1)",Peter V. Brett,4.5,577,28.54,SAR,The stunning debut fantasy novel from author P...,HarperCollins UK,544,"Fiction , Fantasy , Dark Fantasy",9780007287758,English,"Jan 8, 2009",16,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,The Essentials of Finance and Accounting for N...,Edward Fields,NaN,NaN,58.99,SAR,"Based on the bestselling AMA seminar, a nuts-a...",AMACOM,320,"Business &amp, Economics , Accounting , General",9780814416259,English,"Mar 15, 2011",16,20
944,The Magic of Thinking Big,David J. Schwartz,4.6,244,66.87,SAR,The timeless and practical advice in The Magic...,Penguin,256,"Self-Help , Personal Growth , Self-Esteem",9780698198586,English,"Dec 2, 2014",16,20
945,Twas The Nightshift Before Christmas: Festive ...,Adam Kay,4.7,47,41.82,SAR,A short gift book of festive hospital diaries ...,Pan Macmillan,112,"Medical , Health Care Delivery",9781529018592,English,"Oct 17, 2019",16,20
946,Why We Sleep: The New Science of Sleep and Dreams,Matthew Walker,4.8,52,46.85,SAR,'Astonishing ... an amazing book ... absolutel...,Penguin UK,368,"Psychology , Cognitive Psychology &amp, Cognition",9780141983776,English,"Sep 28, 2017",16,20


In [4]:
child_books = pd.read_csv("children_books.csv")
child_books.drop_duplicates(keep='first', inplace=True, ignore_index=True)
child_books.head(5000)

,Title,Author,Desc,Inerest_age,Reading_age
0,The Girl Who Drank the Moon,Kelly Barnhill,"Every year, the evil Protectorate offers a bab...",10-14,10-14
1,Time Between Us,Tamara Ireland Stone,Sixteen-year-old Anna is struggling to underst...,13+,12+
2,Girl Out of Water,Nat Luurtsema,Lou Brown's swimming ambitions sank without tr...,13-18,10+
3,Captive,A J Grainger,Robyn is scared. Ever since the attempted assa...,13+,13
4,The School of Music,Rachel Bowen and Meurig Bowen Illustrator: Dan...,Welcome to the School of Music. In charge is M...,10+,10+
5,The Crystal Stair,Catherine Fisher,The Crystal Stair picks up after At the World’...,12+,8+
6,Dark Inside,Jeyn Roberts,After a huge earthquake strikes every continen...,14+,10+
7,Kite Spirit,Sita Brahmachari,"When her best friend Dawn commits suicide, Kit...",13+,12+
8,I Capture the Castle,Dodie Smith,"Originally published in 1949, this delightful ...",12+,12+
9,Butter,Erin Jade Lange,"Isolated and bullied at school, with a dysfunc...",13+,13+


In [5]:
# we want to replace reading age with appr. age lim
# new columns: upper_age, lower_age
upper_age = []
lower_age = []
for ind in child_books.index: 
    # print(child_books['Reading_age'][1][:2])
    if '-' in child_books['Reading_age'][ind]:
        lower, upper = child_books['Reading_age'][ind].split('-')
        upper_age.append(int(upper))
        lower_age.append(int(lower))
    else: # + 
        lower = int(child_books['Reading_age'][ind][:2].strip(string.punctuation))
        upper = int(lower+3) # manually set 
        upper_age.append(upper)
        lower_age.append(lower)

# add age columsn to child_books
child_books['lower_age'] = lower_age
child_books['upper_age'] = upper_age

In [6]:
child_stories = pd.read_csv('children_stories.Csv')
child_stories.drop_duplicates(keep='first', inplace=True, ignore_index=True)
story_upper = []
story_lower = []
# Strip off age and split by -
for ind in child_stories.index:
    age = child_stories['cats'][ind][4:]
    if '-' in age:
        lower, upper = age.split('-')
        story_lower.append(int(lower))
        story_upper.append(int(upper))
    else: #+
        try: 
            lower = int(age.strip(string.punctuation))
            upper = lower + 3
            story_lower.append(int(lower))
            story_upper.append(int(upper))
        except: # in this case we will default to 2-9 years (default)
            story_lower.append(2)
            story_upper.append(9)
            
# Add 2 columns to child_stories
child_stories['lower_age'] = story_lower
child_stories['upper_age'] = story_upper

# child_stories['Author'] = [None for i in range(len(child_stories))]

In [7]:
# child_books.head()
# child_stories.head(2000)
# We won't be using the 'Reading_age' column 

In [8]:
child_books.columns, child_stories.columns

(Index(['Title', 'Author', 'Desc', 'Inerest_age', 'Reading_age', 'lower_age',
        'upper_age'],
       dtype='object'),
 Index(['names', 'cats', 'desc', 'lower_age', 'upper_age'], dtype='object'))

In [9]:
# is there a way to combine these two dataframes together? 
# we only need the title, upperage, lowerage, desc and author from each dataframe
df_1 = child_books.copy().drop(columns = ['Inerest_age', 'Reading_age'])
df_1['Type'] = 'Books'
df_1 = df_1.rename(columns = {'Title': 'title', 'Desc' : 'desc', 'Author':'author'})

df_2 = child_stories.copy().drop(columns = ['cats']) 
df_2['Type'] = 'Stories'
df_2 = df_2.rename(columns = {'names': 'title', 'Author':'author'})

df_combined = pd.concat([df_1, df_2])
# df_combined.value_counts('title')
# df_combined

In [10]:
# Rename columns of adv_reads 
try: 
    adv_reads.drop(columns=['rating', 'voters','price','currency','publisher','page_count','generes','ISBN','language','published_date'], inplace=True)
except:
    pass
adv_reads = adv_reads.rename(columns = {'description': 'desc'})
adv_reads['Type'] = ['Books' for i in range(len(adv_reads))]
adv_reads.fillna('', inplace=True)
# adv_reads

In [11]:
# get rid of numbers
df_combined['title_formatted'] = df_combined['title'].str.replace('\d+', '')
df_combined['desc_formatted'] = df_combined['desc'].str.replace('\d+', '')
adv_reads['title_formatted'] = adv_reads['title'].str.replace('\d+','')
adv_reads['desc_formatted'] = adv_reads['desc'].str.replace('\d+','')

# make lowercase
df_combined['title_formatted'] = df_combined['title_formatted'].apply(str.lower)
df_combined['desc_formatted'] = df_combined['desc_formatted'].apply(str.lower)
adv_reads['title_formatted'] = adv_reads['title_formatted'].apply(str.lower)
adv_reads['desc_formatted'] = adv_reads['desc_formatted'].apply(str.lower)


<ipython-input-11-cecfd9b579cc>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_combined['title_formatted'] = df_combined['title'].str.replace('\d+', '')
<ipython-input-11-cecfd9b579cc>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_combined['desc_formatted'] = df_combined['desc'].str.replace('\d+', '')
<ipython-input-11-cecfd9b579cc>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  adv_reads['title_formatted'] = adv_reads['title'].str.replace('\d+','')
<ipython-input-11-cecfd9b579cc>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  adv_reads['desc_formatted'] = adv_reads['desc'].str.replace('\d+','')


In [12]:
# We can tokenize the words here to remove punctuation 
# Tokenize words 
nltk.download('punkt')
df_combined['title_desc'] = df_combined['title_formatted'].apply(word_tokenize) + df_combined['desc_formatted'].apply(word_tokenize)
adv_reads['title_desc'] = adv_reads['title_formatted'].apply(word_tokenize) + adv_reads['desc_formatted'].apply(word_tokenize)
# df_combined

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Remove stopwords
nltk.download('stopwords')
def remove_stopwords(input1):
    words = []
    for word in input1:
        if word not in stopwords.words('english'):
            words.append(word)
    return words

df_combined['title_desc'] =  df_combined['title_desc'].apply(remove_stopwords)
adv_reads['title_desc'] =  adv_reads['title_desc'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Lemmatize 
nltk.download('wordnet')
nltk.download('omw-1.4')
lem = WordNetLemmatizer()
def lemma_wordnet(input):
    return [lem.lemmatize(w) for w in input]
df_combined['title_desc'] = df_combined['title_desc'].apply(lemma_wordnet)
adv_reads['title_desc'] = adv_reads['title_desc'].apply(lemma_wordnet)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


In [ ]:
# translate list to string
def join_text(input):
    combined = ' '.join(input)
    return combined
df_combined['title_desc_filtered']=df_combined['title_desc'].apply(join_text)
adv_reads['title_desc_filtered']=adv_reads['title_desc'].apply(join_text)
# adv_reads

In [ ]:
adv_reads['desc'][2]

"Take a globetrotting journey all over the world--and beyond!--with this companion art book to the hit video game for the Nintendo Switch(TM) system!\n\nIn October of 2017, Super Mario Odyssey(TM) took the gaming world by storm. Now, discover the art and expertise that went into creating one of Nintendo's best-loved games!\n\n This full-color volume clocks in at over 350 pages and features concept art, preliminary sketches, and notes from the development team, plus insight into some early ideas that didn't make it into the game itself! Explore the world of Super Mario Odyssey from every angle, including screen shots, marketing material, and more, to fully appreciate this captivating adventure"

In [ ]:
# Remove punctuation for clarity 
df_combined['title_desc_filtered'] = df_combined['title_desc_filtered'].str.replace(r'[^\w\s]+', '')
# df_combined

<ipython-input-17-1c4badf68a31>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_combined['title_desc_filtered'] = df_combined['title_desc_filtered'].str.replace(r'[^\w\s]+', '')


In [ ]:
user_age = '17'

In [ ]:
user_age = int(user_age)
if int(user_age) >= 16: 
    df_16_plus = pd.concat([df_combined, adv_reads])
    df_filter = df_16_plus[(df_16_plus['lower_age'] <= user_age) & (df_16_plus['upper_age'] >= user_age)]
else: 
    df_filter = df_combined[(df_combined['lower_age']<=user_age) & (df_combined['upper_age']>=user_age) ]
# df_filter

In [ ]:
vectorizer = TfidfVectorizer(max_features=1800, lowercase=True, stop_words='english', ngram_range=(1,1)) #unigrams and bigrams
tf_idf_output = vectorizer.fit_transform(df_filter['title_desc_filtered'])
vocab = np.array(vectorizer.get_feature_names())
# vocab

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
num_topics = '25'

In [ ]:
# Perform LDA 
num_topics = int(num_topics) # can be changed 
lda = LatentDirichletAllocation(n_components=num_topics, random_state=1).fit(tf_idf_output)

In [ ]:
num_top_words = '15'

In [ ]:
num_top_words = int(num_top_words) # can be changed. Can also be generated from Cohere
  
topics_set = []

# draft that works
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        topics_set.append(set([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

display_topics(lda, vocab, num_top_words)

In [ ]:
user_phrase = 'small town murder that results in a court case from the victim'

In [ ]:
# word correct
nltk.download('words')
words_corpus = words.words()

user_input_list = user_phrase.split()
user_input_list = [elem.strip(string.punctuation).lower() for elem in user_input_list]

def run_autocorrect(input_word): 
    closest_words = pq()
    for word in words_corpus: 
        distance = editdistance.eval(word, input_word)
        closest_words.put((distance, word))
    return closest_words.get()

new_phrase_list = []
for elem in user_input_list: 
    if elem not in words_corpus: 
        correct_word = run_autocorrect(elem)
        new_phrase_list.append(correct_word[1])
    else: 
        new_phrase_list.append(elem)

user_phrase = " ".join(new_phrase_list)
user_phrase

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


'small town murder that result in a court case from the victim'

In [ ]:
# There is no point of removing stopwords here because we are seeing how much similarity there is regardless. 
# I am thinking of adding spell-correction here? 

# user_phrase = set(user_phrase.lower().split())
# we will preprocess the user phrase just like how we preprocessed the descriptions. Removing stopwords won't matter
# since we are looking at frequency 

user_phrase = word_tokenize(user_phrase)
user_phrase = [word if word not in string.punctuation else '' for word in user_phrase]
 #user_phrase.replace(string.punctuation, '')
user_phrase = set([lem.lemmatize(w).lower() for w in user_phrase])
# print(user_phrase)

best_topic = topics_set[0] #initialization 
best_intersection = -1
for group in topics_set: 
    # print(group, user_phrase, len(user_phrase.intersection(group)))
    print(group)
    if len(user_phrase.intersection(group)) > best_intersection: 
        best_topic = group
        best_intersection = len(user_phrase.intersection(group))

print(best_topic, best_intersection) # returns best list of topics and length of the intersection 

{'dent', 'secret', 'jason', 'arthur', 'universe', 'guide', 'adventure', 'galaxy', 'elio', 'big', 'edition', 'trilogy', 'onestate', 'kinsey', 'hitchhiker'}
{'feminist', 'helen', 'meeny', 'winning', 'greg', 'award', 'talk', 'jump', 'eeny', 'author', 'president', 'diary', 'grace', 'day', 'contemporary'}
{'robert', 'anthony', 'trading', 'step', 'need', 'effective', 'mental', 'control', 'le', 'money', 'game', 'thing', 'platform', 'winterfell', 'day'}
{'kat', 'thrawn', 'leader', 'salander', 'destruction', 'investing', 'war', 'little', 'darksiders', 'stock', 'market', 'vagrant', 'amy', 'blomkvist', 'shining'}
{'emergency', 'tracy', 'business', 'marketing', 'build', 'offering', 'simple', 'successful', 'medicine', 'renowned', 'brian', 'success', 'overcome', 'customer', 'personal'}
{'story', 'memoir', 'weapon', 'map', 'origin', 'force', 'zeinab', 'edition', 'influence', 'witcher', 'holmes', 'sherlock', 'persuasion', 'chaos', 'gangster'}
{'valor', 'escalon', 'series', 'regarding', 'sorcerer', 'fa

In [ ]:
# df_filter

In [ ]:
# Now we need to match the book descriptions to the topic that was chosen 
books = pq() #min heap 
for index, row in df_filter.iterrows():
    # print(row['title_desc_filtered'])
    intersect_len = len(set(row['title_desc_filtered'].split()).intersection(best_topic))
    # print(set(row['title_desc_filtered'].split()))
    books.put((-1*intersect_len, row['title'].strip(string.punctuation), row['author'].strip(string.punctuation), row['Type'].strip(string.punctuation)))


In [ ]:
num_recs = '5'

In [ ]:
books_hm = {}
while len(books_hm) < 5: 
    book = books.get()
    # print(book[0])
    books_hm[book[1]] = (book[2], book[3])

books_hm

{'The Red Signal: An Agatha Christie Short Story': ('Agatha Christie',
  'Books'),
 'The Mysterious Affair at Styles': ('Agatha Christie', 'Books'),
 'A Trace of Crime (a Keri Locke Mystery--Book #4': ('Blake Pierce', 'Books'),
 'A Trace of Vice (a Keri Locke Mystery--Book #3': ('Blake Pierce', 'Books'),
 'Death in an Elegant City: An English Village Murder Mystery': ('Sara Rosett',
  'Books')}

In [ ]:
# cohere API key
# KTQmgyEWSk81jeyS98xCMB1iRuWjZ5KDzSkrdw0b

In [ ]:
theme = 'crime, murder, new york city, suspect, man, FBI, detective'

In [ ]:
import cohere
co = cohere.Client('KTQmgyEWSk81jeyS98xCMB1iRuWjZ5KDzSkrdw0b')
prediction = co.generate(
  model='large',
  prompt='--\nProduct: Book \nKeywords: ' + str(theme) + str('Exciting Book Description:'),
  max_tokens=50,
  temperature=0.8,
  k=0,
  p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop_sequences=["--"],
  return_likelihoods='NONE')
print('Idea: {}...'.format(prediction.generations[0].text))
prediction.generations

Idea: 
In a savage murder in New York City, a man is found dead at the scene of a crime. Hours later, the FBI picks up the unusual phone calls from a suspect in the case.

This is the premise of a thrilling...


[cohere.Generation {
 	text: 
 In a savage murder in New York City, a man is found dead at the scene of a crime. Hours later, the FBI picks up the unusual phone calls from a suspect in the case.
 
 This is the premise of a thrilling
 	likelihood: None
 	token_likelihoods: None
 }]

In [ ]:
send_sms = 'True'

In [ ]:
if send_sms == True: 
    # send to twilio 

IndentationError: expected an indented block (<ipython-input-36-ea0bf1780628>, line 2)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a000e112-1cbe-4ff9-9e6d-cef9a0ebc108' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>